In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# import package
import os
import random
from glob import glob
from warnings import filterwarnings

import argparse

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import cv2
from PIL import Image

import torch
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim

from collections import defaultdict

filterwarnings('ignore') 

In [ ]:
# random seed
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [ ]:
# gpu or cpu
device = 'cuda' if torch.cuda.is_available() else 'cpu'
kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {} 
print(f'device：{device}')

# Load pretrained model

In [ ]:
# Model path
model_path = '/kaggle/input/mnist-mobilenet-v3-earlystopping-pytorch/mobilenet_v3_large.pth'

In [ ]:
# load pretrained model
model = torch.load(model_path)

# Dataset, Dataloader

In [ ]:
# read data
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
print(test.head())
print('number of data: ', test.shape[0])

In [ ]:
# transform
transform = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        #transforms.RandomRotation(degrees=20),
        transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
        ]),
    'val': transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
        ]),
}

In [ ]:
# Dataset
class MNISTDataset(Dataset):
    def __init__(self, df, transform=None, phase=None):
        self.df = df
        self.transform = transform
        self.phase = phase

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        
        # loads the index-th image and preprocesses it
        data = self.df.iloc[index]
        image = data[0:].values.reshape((28,28)).astype(np.uint8)
        image = self.transform[self.phase](image)
        
        # get the index-th label
        label = data[0]

        return image, label

In [ ]:
# Instantiation of Dataset
test_dataset = MNISTDataset(test, transform=transform, phase='val')

In [ ]:
# Instantiation of Dataloader
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Predict test data

In [ ]:
# creation of predictive data frames
preds = []

# Read data from dataloader in mini-batch units
for images, _ in test_dataloader:
    
    # images to device
    images = images.to(device)
    
    # evaluation mode
    model.eval()
    
    # predict
    outputs = model(images)
    pred = torch.argmax(outputs, dim=1)
    pred = pred.to('cpu').numpy()

    # add results to list
    preds.extend(pred)

# Submission

In [ ]:
# read submission format data
sub = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
print(sub.head())
print('number of data: ', sub.shape[0])

In [ ]:
# replace label column
sub['Label'] = preds

# export file
sub.to_csv('submission.csv', index=False)